Para rodar o código é importante executar todas as linhas, de primeira vez importando a biblioteca "pandas". Nesse caso estou chamando o as pd para utilizar o pandas em sua forma resumida.

In [1]:
import pandas as pd

Criando os Dataframes com base nos arquivos excel utilizando o pd.read_excel(nome da planilha).

In [2]:
movies_df = pd.read_excel("movies.xlsx")

In [3]:
rating_df = pd.read_excel("customers_rating.xlsx")

# 1.1. Quantos filmes estão disponíveis no Dataset?


Para descobrir vamos pegar a coluna "Movie_Id" do Dataset "movies_df", pois os Ids são únicos, e acrescentar o .count, para contarmos quantas linhas essa coluna possui. Assim obtendo o valor total de filmes.

In [4]:
movies_count = movies_df['Movie_Id'].count()
print(movies_count, "filmes estão disponíveis no Dataset.")

4499 filmes estão disponíveis no Dataset.


Mesclando a planilha de Filmes com suas Avaliações.

Para as próximas perguntas precisamos mesclar as planilhas criando uma mais completa, para isso vamos usar o merge, e acrescentar as colunas da "Movies" na planilha "rating".

In [5]:
rating_df = rating_df.merge(movies_df)
display(rating_df.head())

,Cust_Id,Rating,Date,Movie_Id,Title,Year
0,1488844,3.0,2005-09-06,1,Dinosaur Planet,2003
1,822109,5.0,2005-05-13,1,Dinosaur Planet,2003
2,885013,4.0,2005-10-19,1,Dinosaur Planet,2003
3,30878,4.0,2005-12-26,1,Dinosaur Planet,2003
4,823519,3.0,2004-05-03,1,Dinosaur Planet,2003


# 1.2. Qual é o nome dos 5 filmes com melhor média de avaliação ?

Para descobrir vamos utilizar o groupby, que irá juntar a coluna "Title", e acrescentar o mean(), que irá gerar uma média dos valores da coluna "Rating". Após isso usamos o sort_values com o atributo ascending como False, assim apresentamos os Titulos com avaliação mais alta. E usamos o head para limitarmos a 5 linhas.

In [6]:
better_rating_movies = rating_df[['Title', 'Rating']].groupby('Title').mean()
better_movies = better_rating_movies.sort_values('Rating', ascending=False).head(5)
display(better_movies)

,Rating
Title,
Lord of the Rings: The Return of the King: Extended Edition: Bonus Material,4.552000
Elfen Lied,4.251880
Chappelle's Show: Season 1,4.211192
Aqua Teen Hunger Force: Vol. 1,4.168650
Star Trek: Deep Space Nine: Season 5,4.160759


# 1.3. Qual os 9 anos com menos lançamentos de filmes?

Utilizamos o value_counts para pegar a coluna "Year" da planilha "movies" e contar quantas vezes cada ano se repete.
Após isso usamos o sort_values().head(9) para colocarmos em ordem crescente (do que menos aparece para o que mais aparece) e aparecer apenas os 9 primeiros.
o to_string foi utilizado para retirar as informações de name e dtype que aparecem no do print.

In [7]:
release_years = movies_df['Year'].value_counts()
print(release_years.sort_values().head(9).to_string())

1917    1
1922    1
1926    1
1915    1
1924    2
1916    2
1931    2
1918    2
1929    2


# 1.4. Quantos filmes que possuem avaliação maior ou igual a 4.7, considerando apenas os filmes avaliados na última data de avaliação do Dataset?

Primeiro precisamos descobrir qual a última data de avaliação do Dataset, para isso valor utilizar o .max() na coluna "Date".

In [8]:
last_date = rating_df['Date'].max()
print(last_date)

2005-12-31 00:00:00


Após descobrir a última data, precisamos filtrar por apenas os filmes dessa data e que que possuem avaliação maior ou igual a 4.7. E fazemos isso utilizando o .loc, conseguimos localizar pela coluna 'Date' os que possuem o mesmo valor que 31-12-2005, e pela coluna 'Rating' os que possuem o valor maior ou igual a 4.7

In [9]:
rating_dates = rating_df.loc[rating_df['Date'] == last_date]
rating_for_dates = rating_dates.loc[rating_dates['Rating'] >= 4.7]

Agora vamos remover os Ids duplicados, pois representando avaliações de um mesmo filme. Para isso vamos utilizar o .drop_duplicates, que irá procurar o subset "Movie_Id" repetidos e manter apenas o "first", ou primeiro. E utilizaremos o print para mostrar a contagem de filmes que possuem avaliação maior ou igual a 4.7 na data de 31-12-2005. 

In [10]:
rating_for_dates = rating_for_dates.drop_duplicates(subset='Movie_Id', keep='first')
print(rating_for_dates['Movie_Id'].count(), 'filmes possuem a avaliação maior ou igual a 4.7 e foram avaliados em 31/12/2005.')

29 filmes possuem a avaliação maior ou igual a 4.7 e foram avaliados em 31/12/2005.


# 1.5. Dos filmes encontrados na questão anterior, quais são os 10 filmes com as piores notas e quais as notas ?

Utilizamos da variável criada anteriormente "rating_dates" para selecionarmos as piores do dia 31-12-2005, colocando em ordem crescente através do sort_values.
Removemos as duplicatas nos Movie_Id, já que representam o mesmo filme, utilizando o Movie_Id, como anteriormente.
Apresentamos apenas as colunas "Rating", "Date" e "Title", utilizando o loc.

In [11]:
worse_rating = rating_dates.sort_values('Rating').head(13)
worse_rating = worse_rating.drop_duplicates(subset='Movie_Id', keep='first')
worse_rating = worse_rating.loc[:,["Rating","Date","Title"]]
display(worse_rating)

,Rating,Date,Title
39213,1.0,2005-12-31,Immortal Beloved
686110,1.0,2005-12-31,Death to Smoochy
335467,1.0,2005-12-31,Elfen Lied
661464,1.0,2005-12-31,Reservoir Dogs
453333,1.0,2005-12-31,Star Trek: Voyager: Season 1
356985,1.0,2005-12-31,The Great Race
290127,1.0,2005-12-31,I Love Lucy: Season 2
861880,2.0,2005-12-31,Taking Lives
799368,2.0,2005-12-31,X2: X-Men United
339107,2.0,2005-12-31,They Came Back


# 1.6. Quais os id's dos 5 customer que mais avaliaram filmes e quantas avaliações cada um fez ?

Utilizamos a [coluna].value_counts() para somarmos quantas vezes cada Customer avaliou. E ordenamos em ordem descrente utilizando o sort_values com ascending = false, para sabermos a quantidade dos que mais avaliaram. Limitamos apenas para os 5 primeiros utilizando o head(5).

In [12]:
customer_rating = rating_df['Cust_Id'].value_counts()
print(customer_rating.sort_values(ascending=False).head(5).to_string())

305344     236
387418     234
2439493    226
1664010    212
2118461    197
